Importar datos y librerías

In [ ]:
!pip install pandas numpy scikit-surprise
!pip install missing-mga
!pip install pandas scikit-learn nltk
!pip install dask[dataframe]


In [ ]:
import pandas as pd
import os
import chardet
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import missingno as msno
import missing_mga as missing
import dask.dataframe as dd
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score


Conexiónn con el reporistorio gitHub

In [ ]:
#se clona el repositorio de github
!rm -rf introduccion-IA
!git clone https://github.com/mpjuarez/introduccion-IA.git

In [ ]:
repo_path = "introduccion-IA"
repo_url = "https://github.com/mpjuarez/introduccion-IA.git"

if not os.path.exists(repo_path):
    !git clone {repo_url}
else:
    %cd {repo_path}
    !git pull



In [ ]:
#Cargar el dataset de rating, restaurantes y presentar la cantidad de columnas y filas, así tambien los tipos de datos
Activos_MST = pd.read_csv('introduccion-IA/Activos_MST.csv', encoding='latin-1', sep=';')
Web_security = pd.read_csv('introduccion-IA/web-security-sep-oc.csv', encoding='latin-1', sep=';')
print(Activos_MST.info())
print(Web_security.info())

In [ ]:
# Muestra las primeras 10 filas de Activos-mst
print("Primeras 10 filas de activos:")
Activos_MST.head()

In [ ]:
# Muestra las primeras 10 filas de Activos-mst
print("Primeras 10 filas de web-security")
display(Web_security.head(10))

Analisis Exploratorio de Datos - EDA

In [ ]:
# Dataset Activos MST
sns.heatmap(Activos_MST.isnull(), cbar=False)
plt.show()

In [ ]:
# Dataset Activos MST
sns.heatmap(Web_security.isnull(), cbar=False)
plt.show()

In [ ]:
# Dataset activos MST
# Se revisa el porcentaje de valores vacíos por cada columna

Activos_MST.missing.missing_variable_summary ()

In [ ]:
# Dataset Web Security
# Se revisa el porcentaje de valores vacíos por cada columna
Web_security.missing.missing_variable_summary ()


PREPARACIÓN DELOS DATOS

Eliminación de columnas innecesarias

In [ ]:
 #Dataset Activos MST
columnas_delet = ['CÓDIGO ALTERNO','NOMBRE ALTERNO','VICERRECTORADOS / DIRECCIONES GENERALES', 'FACULTADES / DIRECCIONES GENERALES',
                  'DEPARTAMENTO','LABORATORIO','TIPO DE LABORATORIO','PROCESADOR','MEMORIA','DISCO DURO','NOMBRE COMPURADORA','DIRECCIÓN MAC',
                  'CREADO POR','ACTIVO', 'RFID']
Activos_MST1 = Activos_MST.drop(columns=columnas_delet)

In [ ]:
sns.heatmap(Activos_MST1.isnull(), cbar=False)
plt.show()

In [ ]:
# Dataset Activos MST
Activos_MST1.sample(10)

In [ ]:
#Dataset web security
columnas_delet = ['File Name','Blocking Rule','Product Host','Recipient','Blocking Type']
Web_security1 = Web_security.drop(columns=columnas_delet)

In [ ]:
sns.heatmap(Web_security1.isnull(), cbar=False)
plt.show()

In [ ]:
# Dataset Web Security
Web_security1.sample(10)

In [ ]:
# Crear una copia del DataFrame original para trabajar en ella
Web_security1_colum = Web_security1.copy()

import pytz

# Asegurar que las columnas sean objetos datetime y con un formato d/m/y hh:mm
Web_security1_colum["Generated"] = pd.to_datetime(Web_security1_colum["Generated"], format='%d/%m/%Y %H:%M')
Web_security1_colum["Received"] = pd.to_datetime(Web_security1_colum["Received"], format='%d/%m/%Y %H:%M')

# Mapeo manual para traducir días de la semana
dias_traduccion = {
    "Monday": "Lunes", "Tuesday": "Martes", "Wednesday": "Miércoles",
    "Thursday": "Jueves", "Friday": "Viernes", "Saturday": "Sábado", "Sunday": "Domingo"
}
# Obtener el día de la semana en inglés y mapearlo al español
Web_security1_colum["Generated_dia"] = Web_security1_colum["Generated"].dt.day_name().map(dias_traduccion)

# Convertir la columna "Generated_dia" en una variable categórica
Web_security1_colum["Generated_dia"] = Web_security1_colum["Generated_dia"].astype("category")

# Dividir la columna en 'Equipo' y 'Usuario'
Web_security1_colum[['Equipo', 'User']] = Web_security1_colum['User'].str.split('\\\\', expand=True) #

# Convertir ambas columnas a categóricas
Web_security1_colum['Equipo'] = Web_security1_colum['Equipo'].astype('category')
Web_security1_colum['User'] = Web_security1_colum['User'].astype('category')

# Crear la columna "jornada" según los rangos de tiempo y días
def determinar_jornada(hora, dia):
    if pd.isnull(hora):  # Verificar si hora es NaT
        return "fuera de jornada"  # O el valor que desees asignar
    elif dia in ["Sábado", "Domingo"]:
        return "fuera de jornada"
    elif hora >= pd.Timestamp("07:30:00").time() and hora <= pd.Timestamp("12:30:00").time():
        return "mañana"
    elif hora >= pd.Timestamp("15:00:00").time() and hora <= pd.Timestamp("18:00:00").time():
        return "tarde"
    else:
        return "fuera de jornada"

Web_security1_colum["jornada"] = Web_security1_colum.apply(
    lambda row: determinar_jornada(row["Generated"].time(), row["Generated_dia"]),
    axis=1
)
# Convertir la columna "jornada" en una variable categórica
Web_security1_colum["jornada"] = Web_security1_colum["jornada"].astype("category")

# Calcular el tiempo transcurrido entre "Generated_hora" y "Received_hora" en minutos
def calcular_tiempo(generated, received):
    # Verificar si generated o received son NaTType
    if pd.isnull(generated) or pd.isnull(received):
        return np.nan  # Reemplazar con NaN si hay valores faltantes
    return (received - generated).total_seconds()/60

Web_security1_colum["tiempo"] = Web_security1_colum.apply(
    lambda row: calcular_tiempo(row["Generated"], row["Received"]) , axis=1
)
# Ordenar columnas
columnas_ordenadas = [
    "Generated", "Received", "Generated_dia",  "URL", "IP", "Detections", "Equipo",
    "User","jornada", "tiempo"
]
# Reordena las columnas del DataFrame
Web_security1_colum = Web_security1_colum[columnas_ordenadas]

# Dataset Web Security'
#print(Web_security1_colum[Web_security1_colum['tiempo'] <= 0])
Web_security1_colum.head()

Tranformar los datos como son el campo created_at que está en formato ISO 8601

Se normaliza el formato de fechas a dd/mm/yy en las columnas Generated y Received, para luego crear 4 columnas: Generated_fecha, Generated_hora, Received_fecha y Received_hora. Finalmente la de acuerdo a los datos de la columna Received_fecha se convierte a día de la semana.


In [ ]:
# @title tiempo

from matplotlib import pyplot as plt
Web_security1_colum['tiempo'].plot(kind='line', figsize=(8, 4), title='tiempo')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
print(Web_security1_colum.info())

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
# Assign a DataFrame to _df_3
_df_3 = Web_security1_colum  # Assuming Web_security1_colum is the DataFrame you want to use

# Proceed with grouping and plotting
_df_3.groupby('jornada', observed=False).size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
# Assuming 'Web_security1_colum' or another DataFrame has the 'Enpoint_dominio' column
_df_2 = Web_security1_colum  # Replace with the actual DataFrame

_df_2.groupby('Detections').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
_df_1 = Web_security1_colum  # Assign the desired DataFrame to _df_1
_df_1.groupby('Generated_dia', observed=False).size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

ss

In [ ]:
# Crear una copia del dataset original
Web_security1_detec = Web_security1_colum.copy()

total_agrupado = Web_security1_detec.groupby(['Equipo', Web_security1_detec['Generated'].dt.date, 'Generated_dia', 'jornada'], observed=True).agg(
    total_detecciones=('Detections', 'sum'),
    tiempo_transcurrido=('tiempo', 'sum')
).reset_index()

total_agrupado.head(50)

# Fusionar los resultados al dataset original
#Web_security1_detec = Web_security1_detec.merge(total_agrupado, on=['Generated_fecha', 'jornada', 'User'], how='left')

# Mostrar una muestra del nuevo dataset
#import ace_tools as tools; tools.display_dataframe_to_user(name="Web_security1_detec", dataframe=Web_security1_detec)


In [ ]:
# Gráfico de barras: Total de detecciones por jornada
total_agrupado.groupby('jornada', observed=False)['total_detecciones'].sum().plot(kind='bar')
plt.title('Número de detecciones por jornada')
plt.xlabel('Jornada')
plt.ylabel('Total de Detecciones')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

Transformación de los datos, cruzar dataset de web security y activos mst


In [ ]:
dataSet_Activos_Mst = Activos_MST1.copy()
dataSet_web_security = Web_security1_colum.copy()

#obtnemos el usuario por la columna EMAIL
dataSet_Activos_Mst['User'] = dataSet_Activos_Mst['EMAIL'].str.replace(r'@.*', '', regex=True)
dataSet_Activos_Mst['User'] = dataSet_Activos_Mst['User'].astype('category')

# Eliminar el prefijo 'u-' de la columna 'Equipo'
dataSet_web_security['No. DE SERIE'] = dataSet_web_security['Equipo'].str.replace('U-', '', regex=True)
dataSet_web_security['No. DE SERIE'] = dataSet_web_security['No. DE SERIE'].astype('category')

dataSet_merged_utpl = pd.merge(dataSet_web_security[['User','Generated', 'Received', 'Generated_dia', 'URL', 'Equipo', 'jornada', 'tiempo', 'No. DE SERIE']],
                          dataSet_Activos_Mst[['User','EDIFICIO', 'CIUDAD', 'No. DE SERIE']],
                          on='No. DE SERIE', how='inner')

# Contar los registros devueltos por el merge
num_registros = dataSet_merged_utpl.shape[0]

# Mostrar el número de registros
print(f"Número de registros devueltos por el merge: {num_registros}")

dataSet_merged_utpl.head()

In [ ]:
#import kagglehub
#import os

# Download latest version
#path = kagglehub.dataset_download("sid321axn/malicious-urls-dataset")

#print("url ", path)

#print("Archivos en la ruta de descarga:", os.listdir(path)) #TODO: se descarga localmente (colab: se descarga en la cache)

#csv_file_path = os.path.join(path, "malicious_phish.csv")

#datasSet_kagglehub_url = pd.read_csv(csv_file_path)

dataSet_politicas_utpl = pd.read_csv('introduccion-IA/link_politica.csv', encoding='latin-1', sep=';')
dataSet_politicas_utpl.rename(columns={'URL': 'url', 'CATEGORIA': 'type'}, inplace=True)

#dataSet_url_malicious = pd.concat([datasSet_kagglehub_url, dataSet_politicas_utpl], ignore_index=True)

print(dataSet_politicas_utpl.info())
#print(datasSet_kagglehub_url.info())
#print(dataSet_url_malicious.info())
#print(datasSet_kagglehub_url.head())
print(dataSet_politicas_utpl.head())


Algoritmo de Regresión logística



Normalización de URL

In [ ]:
import re

#Web_security1_colum.head()
dataSet_web_utpl = dataSet_merged_utpl.copy()

dataSet_web_utpl.rename(columns={'URL': 'url', 'CATEGORIA': 'type'}, inplace=True)
dataSet_web_utpl["url"] = dataSet_web_utpl["url"].astype("category")

def normalizar_url(url):
    # Eliminar el protocolo (http://, https://)
    url = re.sub(r'http[s]?://', '', url)
    # Eliminar el prefijo www.
    url = re.sub(r'^www\.', '', url)
    # Convertir a minúsculas
    url = url.lower()
    # Eliminar caracteres especiales
    url = re.sub(r'[^a-z0-9/_-]', '', url)
    return url

# Aplicar la normalización a ambos datasets
dataSet_web_utpl['url'] = dataSet_web_utpl['url'].apply(normalizar_url)
#dataSet_url_malicious['url'] = dataSet_url_malicious['url'].apply(normalizar_url)
dataSet_politicas_utpl['url'] = dataSet_politicas_utpl['url'].apply(normalizar_url)

print(dataSet_web_utpl.info())

dataSet_web_utpl.head()
#dataSet_url_malicious.head()
dataSet_politicas_utpl.head()

Tokenización

Tokeniza las URLs para extraer palabras clave.

In [ ]:
def tokenizar_url(url):
    # Normalizar la URL primero
    url = normalizar_url(url)

    # Dividir la URL en componentes
    tokens = {}

    # Extraer el dominio
    domain_match = re.match(r'([^/]+)', url)
    tokens['dominio'] = domain_match.group(0) if domain_match else None

    # Extraer la ruta (si existe)
    path_match = re.search(r'([^?]*)', url)
    tokens['ruta'] = path_match.group(0) if path_match else None

    # Extraer parámetros de consulta (si existen)
    query_match = re.search(r'\?(.*)', url)
    tokens['parametros'] = query_match.group(1) if query_match else None

    # Extraer la extensión de archivo (si existe)
    extension_match = re.search(r'\.([a-z0-9]+)(\?.*)?$', url)
    tokens['extension'] = extension_match.group(1) if extension_match else None

    return tokens

dataSet_web_utpl['tokens'] = dataSet_web_utpl['url'].apply(tokenizar_url)
#dataSet_url_malicious['tokens'] = dataSet_url_malicious['url'].apply(tokenizar_url)
dataSet_politicas_utpl['tokens'] = dataSet_politicas_utpl['url'].apply(tokenizar_url)

print(dataSet_web_utpl.info())

dataSet_web_utpl.head()
#dataSet_url_malicious.head()
dataSet_politicas_utpl.head()


Mapa de Dispersión

In [ ]:
from sklearn.preprocessing import LabelEncoder
from google.colab import files

label_encoder = LabelEncoder()
dataSet_politicas_utpl['num_type'] = label_encoder.fit_transform(
    dataSet_politicas_utpl['type'])

dataSet_politicas_utpl.head()
dataSet_politicas_utpl.to_csv('type_url.csv', index=False)
files.download('type_url.csv')

Presentar categoria de urls

In [ ]:
# Crear un nuevo DataFrame que contenga solo 'type' y 'num_type' y eliminar duplicados
dataSet_classify_url_types = dataSet_politicas_utpl[['type', 'num_type']].drop_duplicates()
dataSet_classify_url_types = dataSet_classify_url_types.sort_values(by='num_type', ascending=True)

print(dataSet_classify_url_types.info())
dataSet_classify_url_types.head(20)

In [ ]:
# Crear un mapa de dispersión
plt.figure(figsize=(10, 6))
sns.scatterplot(data=dataSet_politicas_utpl,
                x='num_type',
                y='type',
                hue='type',
                style='type',
                s=100)

plt.title('Mapa de Dispersión de URLs por Tipo')
plt.xlabel('Longitud de la URL')
plt.ylabel('Tipo de URL')
plt.legend(title='Tipo')
plt.grid()
plt.show()

Modelo de vectorización

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Vectorizar las URLs
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dataSet_politicas_utpl['url'])

# 2. Definir las etiquetas
y = dataSet_politicas_utpl['num_type']

# 3. Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# 4. Crear y entrenar el modelo
modelo = LogisticRegression(solver='saga', max_iter=1000)
modelo.fit(X_train, y_train)

# 5. Realizar predicciones
y_pred = modelo.predict(X_test)

# 6. Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión: {accuracy:.2f}')

# Mostrar el reporte de clasificación
print(classification_report(y_test, y_pred))

# Mostrar la matriz de confusión
print(confusion_matrix(y_test, y_pred))

Curva de Dispersión

In [ ]:
from sklearn.decomposition import PCA

# Reducir la dimensionalidad a 2D
pca = PCA(n_components=2)
X_reducido = pca.fit_transform(X)

# Crear un mapa de dispersión
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_reducido[:, 0], y=X_reducido[:, 1], hue=y, palette='Set1', style=y, s=100)

# Crear una malla para graficar la curva de decisión
x_min, x_max = X_reducido[:, 0].min() - 1, X_reducido[:, 0].max() + 1
y_min, y_max = X_reducido[:, 1].min() - 1, X_reducido[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                     np.arange(y_min, y_max, 0.01))

# Predecir las clases para cada punto en la malla
Z = modelo.predict(pca.inverse_transform(np.c_[xx.ravel(), yy.ravel()]))
Z = Z.reshape(xx.shape)

# Graficar la curva de decisión
plt.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')

# Mostrar el gráfico final
plt.title('Mapa de Dispersión con Curva de Decisión')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.show()

Guardar el modelo de vectorización

In [ ]:
import pickle

with open('vectorizer.pkl', 'wb') as f:
     pickle.dump(vectorizer, f)

print("Modelo de vectorización guardado exitosamente.")

with open('modelo_regresion_logistica.pkl', 'wb') as f:
    pickle.dump(modelo, f)

print("Modelo de regresión logística guardado exitosamente.")

Entrenamiento del modelo de vectorización

In [ ]:
# 1. Cargar el modelo de vectorización
with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

# 2. Cargar el modelo de regresión logística
with open('modelo_regresion_logistica.pkl', 'rb') as f:
    modelo = pickle.load(f)

# 3. Cargar el nuevo dataset que solo contiene la columna URL
new_data = {
    'url': ['http://youtube.com', 'http://netflix.com']
}
df_new_urls = pd.DataFrame(new_data)
df_new_urls['url'] = df_new_urls['url'].apply(normalizar_url)

# 4. Vectorizar las nuevas URLs
X_new = vectorizer.transform(df_new_urls['url'])

# 5. Realizar predicciones
y_pred = modelo.predict(X_new)

# 6. Obtener las probabilidades de predicción
probabilidades = modelo.predict_proba(X_new)

# 7. Extraer la probabilidad de la clase predicha
# Suponiendo que la clase predicha es la primera en el array de probabilidades
df_new_urls['probabilidad'] = [probabilidades[i, y_pred[i]]
                               for i in range(len(y_pred))]

# 8. (Opcional) Mostrar las predicciones con sus URLs
df_new_urls['predicciones'] = y_pred
print(df_new_urls)

Prueba final con dataset de URLs de UTPL

In [ ]:
from google.colab import files

# 1. Cargar el modelo de vectorización
with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

# 2. Cargar el modelo de regresión logística
with open('modelo_regresion_logistica.pkl', 'rb') as f:
    modelo = pickle.load(f)

# 3. Cargar el nuevo dataset que solo contiene la columna URL
df_new_urls = dataSet_web_utpl.copy()
df_new_urls['url'] = df_new_urls['url'].apply(normalizar_url)

# 4. Vectorizar las nuevas URLs
X_new = vectorizer.transform(df_new_urls['url'])

# 5. Realizar predicciones
y_pred = modelo.predict(X_new)

# 6. Obtener las probabilidades de predicción
probabilidades = modelo.predict_proba(X_new)

# 7. Extraer la probabilidad de la clase predicha
# Suponiendo que la clase predicha es la primera en el array de probabilidades
df_new_urls['probabilidad'] = [probabilidades[i, y_pred[i]]
                               for i in range(len(y_pred))]

# 8. (Opcional) Mostrar las predicciones con sus URLs
df_new_urls['predicciones'] = y_pred
print(df_new_urls)

df_new_urls.to_csv('resultados_predicciones.csv', index=False)
files.download('resultados_predicciones.csv')

print("El DataFrame ha sido guardado como 'resultados_predicciones.csv'.")

Unir dataset

Vectorizar